# Where, O, Where Do The College Grads Go?
## As early career professionals, graduating students have curiousity about where they may want to move. Where they want is only part of the conversation. In this project, we explore the relationship between city based livable wages and salary data from a set of majors. A recommendation system is built based on cosine similarity of data about overall career salaries. The recommendation offers the 5 most similar wage patterns to Computer Science.


In [107]:
# Package imports
import pandas as pd
import numpy as np

In [108]:
# Data Preprocessing

df_major_salary = pd.read_csv('cp_major_sal.csv')

df_degrees_over_career = pd.read_csv('degrees-that-pay-back.csv')

lis = 'Connecticut, Maine, Massachusetts, New Hampshire, Rhode Island, Vermont, New Jersey, New York, Pennsylvania, Illinois, Indiana, Michigan, Ohio, Wisconsin, Iowa, Kansas, Minnesota, Missouri,  Nebraska, North Dakota, South Dakota, Delaware, District of Columbia, Florida, Georgia, Maryland, North Carolina, South Carolina, Virginia, West Virginia, Alabama, Kentucky, Mississippi, Tennessee, Arkansas, Louisiana, Oklahoma, Texas, Arizona, Colorado, Idaho, Montana, Nevada, New Mexico, Utah, Wyoming, Alaska, California, Hawaii, Oregon, Washington'
my_states = lis.split(', ')
my_states =  [*set(my_states)]
my_states


df_living_wage = pd.read_csv('livingwage.csv')
df_living_wage = df_living_wage[['city','state','one_adult_no_kids_living_wage', 'land_area_sqmi']]
# On average, a full-time employee in the United Stats works 1,801 hours per year, or 37.5 hours per week, 
# which is more than other OECD countries. Source: https://clockify.me/working-hours#:~:text=On%20
# average%2C%20a%20full%2Dtime%20employee%20in%20the%20United%20Stats,more%20than%20other%20OECD%20countries.
df_living_wage['one_adult_no_kids_living_salary'] = df_living_wage['one_adult_no_kids_living_wage']*1801


In [109]:
df_homes = pd.read_csv('united_states_2022.csv')
df_homes = df_homes[['price','addressState','beds','baths','area', 'zestimate', 'homeType']]
df_homes = df_homes.dropna()
df_homes['addressState'].value_counts()

states = {
    'AK': 'Alaska',
    'AL': 'Alabama',
    'AR': 'Arkansas',
    'AZ': 'Arizona',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'IA': 'Iowa',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'MA': 'Massachusetts',
    'MD': 'Maryland',
    'ME': 'Maine',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MO': 'Missouri',
    'MS': 'Mississippi',
    'MT': 'Montana',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'NE': 'Nebraska',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NV': 'Nevada',
    'NY': 'New York',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VA': 'Virginia',
    'VT': 'Vermont',
    'WA': 'Washington',
    'WI': 'Wisconsin',
    'WV': 'West Virginia',
    'WY': 'Wyoming'
}


morts = {
    'AK': 2060,
    'AL': 1171,
    'AR': 1335,
    'AZ': 1516,
    'CA': 2060,
    'CO': 1516,
    'CT': 1735,
    'DC': 1396,
    'DE': 1396,
    'FL': 1396,
    'GA': 1396,
    'HI': 2060,
    'IA': 1209,
    'ID': 1516,
    'IL': 1207,
    'IN': 1207,
    'KS': 1209,
    'KY': 1171,
    'LA': 1335,
    'MA': 1735,
    'MD': 1396,
    'ME': 1735,
    'MI': 1207,
    'MN': 1209,
    'MO': 1209,
    'MS': 1171,
    'MT': 1516,
    'NC': 1396,
    'ND': 1209,
    'NE': 1209,
    'NH': 1735,
    'NJ': 1657,
    'NM': 1516,
    'NV': 1516,
    'NY': 1657,
    'OH': 1207,
    'OK': 1335,
    'OR': 2060,
    'PA': 1657,
    'RI': 1735,
    'SC': 1396,
    'SD': 1209,
    'TN': 1171,
    'TX': 1335,
    'UT': 1516,
    'VA': 1396,
    'VT': 1735,
    'WA': 2060,
    'WI': 1207,
    'WV': 1396,
    'WY': 1516
}
df_homes['State'] = df_homes['addressState'].map(states)
df_homes['Average Mortgage'] = df_homes['addressState'].map(morts)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,10,15,16,17,18,19,21,23,25,26,27,30,31,32,33,34,37,38,40,41,48,51,52,53,54,56,57,58,60,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
<ipython-input-109-7f59dce79d0e>:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_homes['State'] = df_homes['addressState'].map(states)
<ipython-input-109-7f59dce79d0e>:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [110]:
df = df_living_wage.merge(df_homes,  left_on='state', right_on='State')

In [111]:
df

,city,state,one_adult_no_kids_living_wage,land_area_sqmi,one_adult_no_kids_living_salary,price,addressState,beds,baths,area,zestimate,homeType,State,Average Mortgage
0,New York,New York,20.00,300.5,36020.00,"$675,000",NY,4.0,3.0,2444.0,675000.0,SINGLE_FAMILY,New York,1657.0
1,New York,New York,20.00,300.5,36020.00,"$739,000",NY,3.0,3.0,1583.0,671000.0,SINGLE_FAMILY,New York,1657.0
2,New York,New York,20.00,300.5,36020.00,"$160,000",NY,1.0,1.0,750.0,459800.0,SINGLE_FAMILY,New York,1657.0
3,New York,New York,20.00,300.5,36020.00,"$749,999",NY,4.0,3.0,2200.0,789700.0,MULTI_FAMILY,New York,1657.0
4,New York,New York,20.00,300.5,36020.00,"$439,000",NY,3.0,2.0,1910.0,439000.0,MULTI_FAMILY,New York,1657.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459477,Boise,Idaho,14.12,84.0,25430.12,"$515,900",ID,3.0,2.0,1450.0,478800.0,SINGLE_FAMILY,Idaho,1516.0
459478,Boise,Idaho,14.12,84.0,25430.12,"$450,000",ID,2.0,3.0,1552.0,446800.0,TOWNHOUSE,Idaho,1516.0
459479,Boise,Idaho,14.12,84.0,25430.12,"$2,050,000",ID,4.0,5.0,4760.0,1922500.0,SINGLE_FAMILY,Idaho,1516.0
459480,Boise,Idaho,14.12,84.0,25430.12,"$1,269,000",ID,2.0,2.0,2120.0,1244500.0,SINGLE_FAMILY,Idaho,1516.0


In [112]:
df_degrees_over_career = df_degrees_over_career.set_index('Undergraduate Major')


In [113]:
df_degrees_over_career

,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
Undergraduate Major,,,,,,,
Accounting,"$46,000.00","$77,100.00",67.6,"$42,200.00","$56,100.00","$108,000.00","$152,000.00"
Aerospace Engineering,"$57,700.00","$101,000.00",75.0,"$64,300.00","$82,100.00","$127,000.00","$161,000.00"
Agriculture,"$42,600.00","$71,900.00",68.8,"$36,300.00","$52,100.00","$96,300.00","$150,000.00"
Anthropology,"$36,800.00","$61,500.00",67.1,"$33,800.00","$45,500.00","$89,300.00","$138,000.00"
Architecture,"$41,600.00","$76,800.00",84.6,"$50,600.00","$62,200.00","$97,000.00","$136,000.00"
Art History,"$35,800.00","$64,900.00",81.3,"$28,800.00","$42,200.00","$87,400.00","$125,000.00"
Biology,"$38,800.00","$64,800.00",67.0,"$36,900.00","$47,400.00","$94,500.00","$135,000.00"
Business Management,"$43,000.00","$72,100.00",67.7,"$38,800.00","$51,500.00","$102,000.00","$147,000.00"
Chemical Engineering,"$63,200.00","$107,000.00",69.3,"$71,900.00","$87,300.00","$143,000.00","$194,000.00"


In [114]:
for col in list(df_degrees_over_career):
    if col != 'Percent change from Starting to Mid-Career Salary':
        df_degrees_over_career[col] = df_degrees_over_career[col].str.strip('$')
        df_degrees_over_career[col] = df_degrees_over_career[col].str.replace(',', '')
    df_degrees_over_career[col] = df_degrees_over_career[col].astype(float)

## With respect to careers, the dataset below uses salary data from the last 10 years in order to get a perspective on mid career salaries.

In [115]:
# 5 Majors Closest to Accounting Wage Wise For Career Estimates -- Computer Science
x = df_degrees_over_career.iloc[13]
db = df_degrees_over_career.drop(df_degrees_over_career.index[13]) # Make database of all users except user 1
sims = db.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
sorted_sims = sims.sort_values()[::-1]
N=5
userIds = sorted_sims.dropna().iloc[:N].index
df_closest2010 = df_degrees_over_career.loc[userIds]
df_closest2010

,Starting Median Salary,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
Undergraduate Major,,,,,,,
Aerospace Engineering,57700.0,101000.0,75.0,64300.0,82100.0,127000.0,161000.0
Health Care Administration,38800.0,60600.0,56.2,34600.0,45600.0,78800.0,101000.0
Computer Engineering,61400.0,105000.0,71.0,66100.0,84100.0,135000.0,162000.0
Electrical Engineering,60900.0,103000.0,69.1,69300.0,83800.0,130000.0,168000.0
Information Technology (IT),49100.0,74800.0,52.3,44500.0,56700.0,96700.0,129000.0


In [116]:
# 5 Majors Closest to Accounting Wage Wise For Early Career Estimates -- Computer Science
ec_df = df_degrees_over_career.drop(['Mid-Career Median Salary'], axis=1)

x = ec_df.iloc[13]
db = ec_df.drop(df_degrees_over_career.index[13]) # Make database of all users except user 1
sims = db.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
sorted_sims = sims.sort_values()[::-1]
N=5
userIds = sorted_sims.dropna().iloc[:N].index
df_closest = ec_df.loc[userIds]
df_closest

,Starting Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
Undergraduate Major,,,,,,
Aerospace Engineering,57700.0,75.0,64300.0,82100.0,127000.0,161000.0
Health Care Administration,38800.0,56.2,34600.0,45600.0,78800.0,101000.0
Computer Engineering,61400.0,71.0,66100.0,84100.0,135000.0,162000.0
Electrical Engineering,60900.0,69.1,69300.0,83800.0,130000.0,168000.0
Information Technology (IT),49100.0,52.3,44500.0,56700.0,96700.0,129000.0


In [117]:
# 5 Majors Closest to Accounting Wage Wise For Late Career Estimates -- Computer Science
ec_df = df_degrees_over_career.drop(['Starting Median Salary'], axis=1)
x = ec_df.iloc[13]
db = ec_df.drop(df_degrees_over_career.index[13]) # Make database of all users except user 1
sims = db.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
sorted_sims = sims.sort_values()[::-1]
N=5
userIds = sorted_sims.dropna().iloc[:N].index
df_closest = ec_df.loc[userIds]
df_closest

,Mid-Career Median Salary,Percent change from Starting to Mid-Career Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
Undergraduate Major,,,,,,
Health Care Administration,60600.0,56.2,34600.0,45600.0,78800.0,101000.0
Aerospace Engineering,101000.0,75.0,64300.0,82100.0,127000.0,161000.0
Computer Engineering,105000.0,71.0,66100.0,84100.0,135000.0,162000.0
Electrical Engineering,103000.0,69.1,69300.0,83800.0,130000.0,168000.0
Information Technology (IT),74800.0,52.3,44500.0,56700.0,96700.0,129000.0


## When building the recommendation system, the overall recommendartions shared Aerospace Engineering, Health Care Administration, Computer Engineering, Electrical Engineering, Information Technology (IT). The conditions of only considering early or mid career salary altered the positions of each.

## These values may have some error with respect to computer science as a field since the career scope has boosted and likely many computer science majors make more now than 10 years ago.

## To see this, let's run some more cosine similarity on more recent Cal Poly graduate data.

In [119]:
df_major_salary = df_major_salary
for col in list(df_major_salary):
    if col == 'Employment Rate\xa0':
        df_major_salary[col] = df_major_salary[col].str.strip('%')
        # df_major_salary[col]  = df_major_salary[col].astype(int)
        # df_major_salary[col] = int(df_major_salary[col])
    elif col in ['Median Debt',	'Median Salary']:
        df_major_salary[col] = df_major_salary[col].str.strip('$')
        df_major_salary[col] = df_major_salary[col].str.replace(',', '')
        # df_major_salary[col] = df_major_salary[col].astype(int)
df_major_salary['Degree Type'].replace('B', 0 ,inplace=True)
df_major_salary['Degree Type'].replace('M', 1 ,inplace=True)
    

In [120]:
df_major_salary2 = df_major_salary.drop(['Field of Study\xa0','Median Debt', 'Salary Score\xa0'], axis=1)
for col in list(df_major_salary2):
    df_major_salary2[col] = df_major_salary2[col].astype(int)

In [121]:
ec_df = df_major_salary2
x = ec_df.iloc[16]
x.values
db = ec_df.drop(ec_df.index[16]) # Make database of all users except user 1
sims = db.apply(lambda y: (y.values*x.values).sum()/(np.sqrt((y**2).sum())*np.sqrt((x**2).sum())),axis=1)
sorted_sims = sims.sort_values()[::-1]
N=5
userIds = sorted_sims.dropna().iloc[:N].index
df_closest2020 = df_major_salary.loc[userIds]
df_closest2020


,Field of Study,Employment Rate,Median Debt,Median Salary,Salary Score,Degree Type
62,"Electrical, Electronics and Communications Eng...",100,Not Reported,104750,78,1
52,"Aerospace, Aeronautical and Astronautical Engi...",92,Not Reported,94410,72,1
17,Computer Engineering,95,21500,95030,97,0
64,Fire Protection,93,Not Reported,93080,76,1
65,Mechanical Engineering,100,Not Reported,99520,96,1


## Looking at more recent data, we see a more coherent group of similar majors career wise. This shows an interesting development of how much CS majors make over the last decade or so, as well as how the career fields have entered a more traditional expectation of income.


<br>

## To test the manual implementation against a package, we have also ran this same recommendation query through a few packages.

In [144]:
import sklearn.metrics.pairwise as sk
x = pd.DataFrame(x).T
similarities = sk.cosine_similarity(x, df_major_salary2)

In [145]:
similarity_vals = list(similarities[0])

In [146]:
skrecs = pd.DataFrame(df_major_salary.iloc[16])
skrecs = skrecs.T

import heapq
input_list = similarity_vals
number_of_elements = 6
inds = []
for val in heapq.nlargest(number_of_elements, input_list):
  inds.append(similarity_vals.index(val))
inds

for i in inds:
  newskrec = pd.DataFrame(df_major_salary.loc[i]).T
  skrecs = pd.concat([skrecs,newskrec])
skrecs = skrecs.drop(index = (16))
skrecs





,Field of Study,Employment Rate,Median Debt,Median Salary,Salary Score,Degree Type
62,"Electrical, Electronics and Communications Eng...",100,Not Reported,104750,78,1
52,"Aerospace, Aeronautical and Astronautical Engi...",92,Not Reported,94410,72,1
17,Computer Engineering,95,21500,95030,97,0
64,Fire Protection,93,Not Reported,93080,76,1
65,Mechanical Engineering,100,Not Reported,99520,96,1


## Via Sklearn, we have validated that the manual recommendations are accurate since we got identical results. Regardless, the recommendations for similar careers based on majors varies drastucally depending on what data an entity is using. In addition, collaborative filtering can use a variety of different distance metrics, and results may not be the same. 


In [171]:
from sklearn.metrics.pairwise import euclidean_distances
ec_df = df_major_salary2
ec_df
x = ec_df.iloc[16]
x
db = ec_df.drop(ec_df.index[16])
db
dist = []
for row in db.index:
  y = db.loc[row]
  p1 = np.sum([(a * a) for a in x])
  p2 = np.sum([(b * b) for b in y])
  p3 = -1 * np.sum([(2 * a*b) for (a, b) in zip(x, y)])
  dist.append(np.sqrt(np.sum(p1 + p2 + p3)))

## getting min indexes
input_list = dist
number_of_elements = 6
inds = []
for val in heapq.nsmallest(number_of_elements, input_list):
  inds.append(dist.index(val))
inds

# Create new DF
eurecs = pd.DataFrame(df_major_salary.iloc[16])
eurecs = eurecs.T

for i in inds:
  neweurec = pd.DataFrame(df_major_salary.loc[i]).T
  eurecs = pd.concat([eurecs,neweurec])
eurecs = eurecs.drop(index = (16))
eurecs


,Field of Study,Employment Rate,Median Debt,Median Salary,Salary Score,Degree Type
61,"Education, General",100,20000,51830,57,1
64,Fire Protection,93,Not Reported,93080,76,1
51,Accounting and Related Services,100,Not Reported,71730,96,1
63,"Engineering, General",90,Not Reported,80020,30,1
53,"Agriculture, General",100,Not Reported,71580,97,1


##  Definitely differs the results. (Perhaps this goes to show how you shouldn't just trust one Buzzfeed Quiz to determine where to move to, because who knows how they calculate that stuff!)

## Either way, according to Baeldung, a data science website, "Some tasks, such as preliminary data analysis, benefit from both metrics; each of them allows the extraction of different insights on the structure of the data. Others, such as text classification, generally function better under Euclidean distances. Some more, such as retrieval of the most similar texts to a given document, generally function better with cosine similarity"

## Both metrics provide valuable insight, but for our purposes, cosine similarity seems to offer a more logical result.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Some example data to display
skrecs.plot.bar(x='Field of Study\xa0', y='Median Salary'])
# x1 = skrecs['Field of Study\xa0']
# y1 = skrecs['Median Salary']
# fig, ax1 = plt.subplots()
# fig.suptitle('Horizontally stacked subplots')
# ax1.bar(x1, y1)
# ax2.plot(x, y)

In [ ]:
# i feel like here would be a good spot to talk about if any of these share clustering 

# then maybe we can loop in housing data, maybe we ask about directions on tuesday?

# with these in mind, if we see a cluster or not, are we seeing these majors cluster in areas they can afford?

# TODO: make some empty cells to do some MAE evaluation, link in clustering, add another component?